In [2]:
import time
import hmac
import requests
import pandas as pd
from datetime import datetime
import urllib.parse
import asyncio

In [3]:
class FTX_Client_Hicham:
    api_endpoint_base = "https://ftx.com/api/"

    def __init__(self,api_key:str,api_secret:str,subaccount:str=None):
        """Indiquer les clés API et éventuellement un subaccount"""
        self._session = requests.Session()
        self._api_key = api_key
        self._api_secret = api_secret
        self._subaccount = subaccount

    def get_open_orders(self):
        ''' Cette fonction permet de récupérer les ordres qui sont actuellement ouverts'''
        url = self.api_endpoint_base + "orders"
        request = requests.Request('GET', url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        if len(df_data)!=0:
            df_data.drop(['clientId','reduceOnly','liquidation','postOnly','ioc',"future","createdAt","avgFillPrice","remainingSize","filledSize"],axis=1,inplace=True) 
            df_data.set_index("id",inplace=True)    
        return df_data

    def get_positions(self):
        """Cette fonction permet de récupérer les positions actuellement ouvertes sur des contrats futures"""
        url = self.api_endpoint_base + "positions"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        return df_data[df_data["size"]>0]

    def cancel_order(self,order_id):
        """ Cette fonction permet d'annuler un ordre s'il n'a pas déjà été exécuté, l'ID de l'ordre est obtenu dans le résultat de la fonction send_order"""
        url = self.api_endpoint_base + "orders/" + str(order_id)
        request = requests.Request('DELETE',url)
        data = self.send_request(request)
        #df_data = pd.DataFrame(data)
        return data


    def get_balances(self):
        """ Cette fonction permet de récupérer le solde du compte"""
        url = self.api_endpoint_base + "wallet/balances"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        return df_data[df_data["total"]>0]

    def get_order_history(self):
        """ Cette fonction permet de récupérer l'historique des ordres passés (sans forcément avoir été exécutés)"""
        url = self.api_endpoint_base + "orders/history"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)     
        return df_data   

    def get_last_n_funding_rates(self,market,n):
        ''' Cette fonction permet de récupérer les n derniers funding rates sur un future perp'''
        url = self.api_endpoint_base + "funding_rates?future=" + market
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        df_data.set_index("time",inplace=True)
        return df_data[:n]['rate'].to_numpy()

    def send_order(self,market:str,side:str,price:str,type:str,size:str):
        """ Cette fonction permet d'envoyer un ordre:
        market : paire (BTC/USD) ou future (BTC-PERP)      
        side: buy ou sell
        price: prix (mettre 0 pour un ordre market)
        type: limit ou market
        size: quantité
        return: renvoie un récapitulatif de l'ordre passé contenant notamment son ID 
        """
        url = self.api_endpoint_base + "orders"
        params = {"market":market,"side":side,"price":price,"type":type,"size":size,"postOnly":True}
        request = requests.Request("POST",url,json=params)
        data = self.send_request(request)
        return data

    def process_response(self,response):
        ''' Cette fonction sert à traiter la réponse du serveur FTX'''
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']

    def send_request(self,request):
        """ Cette fonction sert à envoyer les requêtes au serveur FTX"""
        ts = int(time.time() * 1000)
        prepared = request.prepare()

        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()

        if prepared.body:
            signature_payload += prepared.body

        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        prepared.headers[f'FTX-KEY'] = self._api_key
        prepared.headers[f'FTX-SIGN'] = signature
        prepared.headers[f'FTX-TS'] = str(ts)
        if self._subaccount :
            prepared.headers[f'FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount)

        response = self._session.send(prepared)
        return self.process_response(response) 
    
    def get_live_quote(self,market:str):
        """Permet d'avoir les quotes pour les spots et les futures, par ex: get_quote("BTC/USD") ou get_quote("BTC-PERP")"""
        url = self.api_endpoint_base + "markets/" + market
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame.from_dict(data,orient="index")
        return df_data

    def get_historical_data(self,market:str,resolution:int,number:int):
        """Permet d'avoir les données historiques sur une paire spot ou sur un future, la résolution est à indiquer en secondes"""
        url = self.api_endpoint_base + "markets/" + market + "/candles?resolution=" + str(resolution) + "&limit=" + str(number)
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        df_data['date'] = pd.to_datetime(df_data["time"]/1000,unit='s',origin='unix')
        df_data.drop(['startTime','time'],axis=1,inplace=True)  
        df_data.set_index('date',inplace=True)
        return df_data

    def get_last_n_minutes_future_spot_spread(self,market_fut:str,market_spot:str,n:int):
        """Permet d'avoir le spread entre un future et le spot associé pour les n dernières minutes"""
        fut = self.get_historical_data(market_fut,60,n)
        spot = self.get_historical_data(market_spot,60,n)
        return ((fut-spot)/spot)["close"]

In [67]:
client = FTX_Client_Hicham(api_key,api_secret,"TestApi")

In [93]:
client.get_balances()

,coin,total,free,availableForWithdrawal,availableWithoutBorrow,usdValue,spotBorrow
1,USD,4.988431,4.988431,4.988431,4.988431,4.988431,0.0


In [88]:
client.get_last_n_minutes_future_spot_spread("ETH-PERP","ETH/USD",5)

date
2022-05-25 15:46:00    0.000102
2022-05-25 15:47:00    0.000000
2022-05-25 15:48:00    0.000153
2022-05-25 15:49:00    0.000102
2022-05-25 15:50:00    0.000000
Name: close, dtype: float64

In [101]:
def enter_trade_conditions(spot_market,fut_market):
    """On récupère le spread future spot des 5 dernières minutes et on compare avec le spread actuel"""
    spreads = client.get_last_n_minutes_future_spot_spread(fut_market,spot_market,6)
    SMA_spread = spreads[:5].mean()
    current_spread = spreads[-1]
    if current_spread > 0 and current_spread > SMA_spread : 
        return True 
    else :
        return False

def enter_trade(spot_market,fut_market,amount):

    """amount : Montant de la pos en USD"""
    """seuil : seuil de temps avant de retirer l'ordre s'il n'est pas passé"""

    ask_price_future = client.get_live_quote(fut_market)[0]["ask"]
    bid_price_spot = client.get_live_quote(spot_market)[0]["bid"]

    size_spot = amount/bid_price_spot
    size_fut = amount/ask_price_future

    """On envoie les ordres"""
    spot_order = client.send_order(spot_market,"buy",bid_price_spot,"limit",size_spot)
    chrono = time.time()

    while len(client.get_open_orders())!=0 and time.time()-chrono <= 25.00 : 
        pass

    if time.time()-chrono > 25.00 :
        client.cancel_order(spot_order["id"])
        return "Error"

    fut_order = client.send_order(fut_market,"sell",ask_price_future,"limit",size_fut)

    return size_spot,size_fut

def close_trade_conditions(spot_market,fut_market):
    """On récupère le spread future spot des 5 dernières minutes et on compare avec le spread actuel"""
    spreads = client.get_last_n_minutes_future_spot_spread(fut_market,spot_market,6)
    SMA_spread = spreads[:5].mean()
    current_spread = spreads[-1]
    if current_spread < 0 and current_spread<SMA_spread: 
        return True 
    else :
        return False

def close_trade(spot_market,fut_market,size_spot,size_fut):

    bid_price_future = client.get_live_quote(fut_market)[0]["bid"]
    ask_price_spot = client.get_live_quote(spot_market)[0]["ask"]

    fut_order = client.send_order(fut_market,"buy",ask_price_spot,"limit",size_fut)
    spot_order = client.send_order(spot_market,"sell",bid_price_future,"limit",size_spot)
    
    return "OK"

In [102]:
spot_market = "ETH/USD"
fut_market = "ETH-PERP"

currently_in_trade = False

while True:

    if enter_trade_conditions(spot_market,fut_market) == True and currently_in_trade == False:
        size_spot,size_fut = enter_trade(spot_market,fut_market,5)
        currently_in_trade = True
    
    if close_trade_conditions(spot_market,fut_market) == True and currently_in_trade == True :
        close_trade(spot_market,fut_market,size_spot,size_fut)
        currently_in_trade = False



KeyboardInterrupt: 